# Metaheuristics for Optimizing Voter Distributions

## Installing and importing
We first install the necessary packages. If you are using Google Colab (or most other web notebooks), you may install the necessary packages here: 

In [ ]:
!pip install bitarray
!pip install gerrychain

We then import all necessary packages: 

In [ ]:
import pickle
import numpy as np
import pandas as pd
from gerrychain.random import random
import math
import time
import statistics
from bitarray import bitarray
from gerrychain import Graph, Partition, Election, grid, MarkovChain
from gerrychain.constraints import single_flip_contiguous, contiguous
from gerrychain.proposals import propose_random_flip, propose_chunk_flip, spectral_recom, recom
from gerrychain.accept import always_accept
from gerrychain.tree import recursive_tree_part
from tqdm import tqdm, trange, tnrange, tqdm_notebook

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('paper')
sns.set(style="whitegrid")
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas

## Defining Constants
We want to define some constants we care about. This basically gives the size of the grid to be $clen \times rlen$, with $clen$ number of districts with $rlen$ blocks each. 

In [ ]:
clen = 5
rlen = 5
num_blocks = clen * rlen

## Working with BitArrays
Voter distributions ($\Delta$) are stored as BitArrays (or arrays of BitArrays). They are the least computationally intensive and memory taxing way to test these, and allow us to use XOR (^) later instead of int equality (==), which also saves a lot of time. 

In [ ]:
def n_bitarray(n):
    """
    Creates a bit array of length num_blocks with n 'true' values. 
    """
    ar = bitarray()
    for i in range(n):
        ar.append(True)
    for i in range(num_blocks - n):
        ar.append(False)
    return ar

def sq_array(ar):
    """
    Converts a bitarray into
    a rectangular array
    """
    sq_ar = [bitarray() for i in range(rlen)]
    for i in range(rlen):
        sq_ar[i] = ar[(clen * i):(clen * (i + 1))]
    return sq_ar

def get_bitarray(delta):
    """
    Converts a sq/rectangular array
    into a bitarray
    """
    output = bitarray()
    for row in delta:
        for entry in row:
            output.append(entry)
    return output

def print_ar(ar):
    """
    Prints a rectangular array
    """
    print("Grid: ")
    for row in ar: 
        rowtext = ""
        for box in row:
            rowtext += "X" if box else "O"
            rowtext += " "
        print(rowtext)
    print()

def to_N(delta):
    """
    Stores a rectangular array as some index
    (written in base two is the original bitarray)
    """
    i = 0
    for r in reversed(range(rlen)):
        for c in reversed(range(clen)):
            i = (i << 1) | delta[r][c]
    return i

def bitarray_to_N(bar):
    """
    The same as above but for a bitarray and not
    a rectangular array
    """
    i = 0
    for dig in bar: 
        i = (i << 1) | dig
    return i

## GerryChains
Working with MGGG's GerryChain to create functions to run MCMC (The Eval function). 

So sometimes there's this fatal crash when trying to generate a base partition to begin the MCMC on, which is really frustrating. I can't seem to figure out what's causing it. So, I'll generate a predefined array of partitions to start with and every time, the MCMC algorithm picks one randomly. This might be due to some memory leak or flooding from the GerryChain algorithm. For the moment, it's as good as generating a new one every time. 

In [ ]:
def assignment_delta(clen, rlen):
    asgmt_grid = grid.Grid((clen, rlen))
    asgmt = recursive_tree_part(graph = asgmt_grid.graph, 
                        parts = range(clen), 
                        pop_target = rlen, 
                        pop_col = 'area', 
                        epsilon = 0.02,
                        node_repeats = 1)
    return asgmt

Run the following code **once** and only once. It should be able to load several 'assignments' into an array of different initial partitions. 

In [ ]:
partitions = []
for i in range(128):
    partitions.append(assignment_delta(clen, rlen))

The following sets up the GerryChain MCMC that we use as our $\mathsf{Eval}$ function. 

In [ ]:
def vote_arrays(delta):
    """
    Takes a BitArray Delta (rectangular grid), 
    and outputs a dictionary of which coordinates
    contain hearts tiles. 
    """
    hearts_array = {}
    unhearts_array = {}
    for row in range(rlen):
        for col in range(clen):
            if delta[row][col]:
                hearts_array[(col, row)] = 1
                unhearts_array[(col, row)] = 0
            else: 
                hearts_array[(col, row)] = 0
                unhearts_array[(col, row)] = 1
    return {'hearts': hearts_array, 'unhearts': unhearts_array}

def mcmc(delta, steps):
    """
    Runs a step number of MCMC on a given delta. 
    Returns the distribution of seats won. 
    """
    vote_list = vote_arrays(delta)
    hearts_list = vote_list['hearts']
    unhearts_list = vote_list['unhearts']

    election = Election("Elc", 
                        {"Hearts": hearts_list, 
                         "Unhearts": unhearts_list}
    )

    ideal_population = rlen

    proposal = partial(recom,
                       pop_col='area',
                       pop_target=ideal_population,
                       epsilon=0.01,
                       node_repeats=2
                      )

    # asgmt = assignment_delta(clen, rlen)

    grd = grid.Grid(dimensions = (clen, rlen), 
                    assignment = partitions[random.randint(0, 127)], 
                  # Above is where asgmt would have gone, but due to the unknown bug 
                  # it just searches a pregenerated array of possibilities. 
                    updaters = {"Elc": election})

    chain = MarkovChain(
        proposal=proposal,
        constraints=[contiguous],
        accept=always_accept,
        initial_state=grd,
        total_steps=steps
    )

    results = [0 for i in range(rlen + 1)]
    for partition in chain:
        results[partition["Elc"].wins("Hearts")] += 1
    
    return results

def evaluate(delta, times):
    """
    Evaluates a delta a number of times, and returns the average
    """
    result = mcmc(delta, times)
    n = 0
    sum = 0
    for idx in range(len(result)):
        num_idx = result[idx]
        n += num_idx
        sum += num_idx * idx
    output = sum / n
    print(output)
    return output

## Metaheuristics/Optimizers
We first define some of the number of times we want to run our optimizers (and whether we want it to show progress):

In [ ]:
def random_step(n):
    """
    Gives a random delta with NumH = n
    """
    a = n_bitarray(n)
    random.shuffle(a)
    return sq_array(a)

def random_sample(times, n):
    """
    Evaluates a times number of random_steps
    (random deltas) and returns a dictionary
    formatted {score: N(index)}
    """
    samp = {}
    for i in tnrange(times, desc = 'Random Sample', leave = False, disable = disable_tqdm):
        step = random_step(n)
        samp[evaluate(step, mcmc_steps)] = to_N(step)
    return samp

def mult_random_sample(times, k_max, n):
    """
    Does mult_random_sample times times and
    gives a list of all the maximums of the
    random_sample (since that is what we're
    interested in)
    """
    maxs = []
    for i in tnrange(times, desc = 'Multiple Random Sample', disable = disable_tqdm):
        run = random_sample(k_max, n)
        maxs.append((max(run), run[max(run)]))
    return maxs

### Random Sampling
The following generates random Deltas and evaluates them. 

In [ ]:
def random_step(n):
    """
    Gives a random delta with NumH = n
    """
    a = n_bitarray(n)
    random.shuffle(a)
    return sq_array(a)

def random_sample(times, n):
    """
    Evaluates a times number of random_steps
    (random deltas) and returns a dictionary
    formatted {score: N(index)}
    """
    samp = {}
    for i in tnrange(times, desc = 'Random Sample', leave = False, disable = disable_tqdm):
        step = random_step(n)
        samp[evaluate(step, mcmc_steps)] = to_N(step)
    return samp

def mult_random_sample(times, k_max, n):
    """
    Does mult_random_sample times times and
    gives a list of all the maximums of the
    random_sample (since that is what we're
    interested in)
    """
    maxs = []
    for i in tnrange(times, desc = 'Multiple Random Sample', disable = disable_tqdm):
        run = random_sample(k_max, n)
        maxs.append((max(run), run[max(run)]))
    return maxs

In [ ]:
samp = mult_random_sample(samp_size, 200, 10)
for run in samp: 
    print(run)

### Shotgun Greedy (Random-Restart Iterated Local Search) Algorithm
The following performs a RRILS optimization, which relies on a cellular automata evolutionary algorithm. 

In [ ]:
# Threshold for Cellular Automata: 
threshold = 0.6

def unhappy(delta):
    """
    returns a tuple of
    (list of coords of unhappy tiles, 
    list of values of unhappy tiles, 
    Clus, ClusH)
    """
    unhappy_tiles = []
    vals = bitarray()
    total_con = [0, 0]
    con = [0, 0]
    for row in range(rlen):
        for col in range(clen):
            box = delta[row][col]
            total_box = 0
            same_box = 0
            for dx, dy in [(1, 0), (-1, 0), (0, -1), (0, 1)]:
                r = row
                c = col
                nr = r + dx
                nc = c + dy
                if 0 <= nr < rlen and 0 <= nc < clen:
                    total_con[delta[r][c]] += 1
                    total_box += 1
                    samity = 0 if box ^ delta[nr][nc] else 1
                    same_box += samity
                    con[delta[r][c]] += samity
            if same_box / total_box < threshold:
                unhappy_tiles.append((r, c))
                vals.append(delta[r][c])
    return {'coords': unhappy_tiles, 
            'vals': vals, 
            'Clus': (con[0] + con[1]) / (total_con[0] + total_con[1]), 
            'ClusH': con[1] / total_con[1]}

def step(unhappy_coords_shuffled, unhappy_list, delta):
    """
    Makes a step with delta and given values of delta
    and returns a new delta
    """
    idx = 0
    ndelta = delta
    for r, c in unhappy_coords_shuffled:
        ndelta[r][c] = unhappy_list[idx]
        idx += 1
    return ndelta

def greedy_step(delta):
    """
    Runs one evolutionary step, gets rid of the
    other parameters in step
    """
    unh = unhappy(delta)
    random.shuffle(unh['coords'])
    return step(unh['coords'], unh['vals'], delta)

def greedy_seq(n, mcmc_steps):
    """
    Runs a greedy sequence multiple times, until
    no more meaningful evolutions can be done. 
    """
    dt = {}
    seed = n_bitarray(n)
    random.shuffle(seed)
    delta = sq_array(seed)
    dt[evaluate(delta, mcmc_steps)] = to_N(delta)
    laststep = to_N(delta)
    k = 0
    while True:
        delta = greedy_step(delta)
        k += 1
        delta_idx = to_N(delta)
        score = evaluate(delta, mcmc_steps)
        if delta_idx == laststep:
            break
        dt[score] = delta_idx
        laststep = delta_idx
    return (dt, k)

def shotgun_greedy(k_max, n, mcmc_steps):
    """
    Runs a greedy sequence multiple times, until
    it has been evaluated k_max times. 
    Returns the full dictionary of
    {score: N(index)}
    """
    sample = {}
    times_run = 0
    pbar = tqdm_notebook(total=k_max, desc = 'Shotgun Greedy', leave = False, disable = disable_tqdm)
    while times_run < k_max: 
        run = greedy_seq(n, mcmc_steps)
        sample.update(run[0])
        times_run += run[1]
        pbar.update(run[1])
    pbar.close()
    return sample

def mult_shotgun_greedy(times, k_max, n, mcmc_steps):
    """
    Runs shotgun_greedy multiple times, 
    and returns a list of
    (max, N that gives the max)
    """
    maxs = []
    for i in tnrange(times, desc = 'Multiple Shotgun Greedy', disable = disable_tqdm):
        run = shotgun_greedy(k_max, n, mcmc_steps)
        maxs.append((max(run), run[max(run)]))
    return maxs

In [ ]:
samp = mult_shotgun_greedy(samp_size, 200, 10, mcmc_steps)
for run in samp: 
    print(run)

### Simulated Annealing
This is the simulated annealing algorithm, which combines a bit of all the previous algorithms we've used. 

We first define a probability of acceptance function: 

In [ ]:
def prob_accept(change_e, temp):
    try: 
        return 1 / (1 + math.exp(-change_e / temp))
    except OverflowError:
        return 0

`random_accept_thresh` gives the probability that a random state is accepted. 

`mcmc_steps` gives the number of mcmc steps to perform.

`temp_initial` gives the initial temperature.

`temp_ratio` is the cooling schedule.

`simulated_anneal` runs the annealing steps as documented. 

`simulated_anneal_random` does simulated annealing but with random progressions (instead of cellular automata progressions). 

In [ ]:
random_accept_thresh = 0.4
temp_initial = 8
temp_ratio = 0.96
heating_ratio = 1.1

# Simulated Annealing algorithm with greedy chance
def simulated_anneal(k_max, n):
    """
    Runs a simulated annealing step
    as described in the paper/documentation
    """
    delta = random_step(n)
    samp = {}
    temp = temp_initial
    eval_now = evaluate(delta, mcmc_steps)
    ra_thresh = random_accept_thresh
    k = 0
    pbar = tqdm_notebook(total=k_max, desc = 'Simulated Anneal', leave = False, disable = disable_tqdm)
    while k < k_max:
        samp[eval_now] = to_N(delta)
        delta_n = greedy_step(delta)
        eval_new = evaluate(delta_n, mcmc_steps)
        k += 1
        pbar.update(1)
        change_e = eval_new - eval_now
        # we can also give the change as a proportion, which makes hyperparameter optimization
        # better for generalization: 
        prop_change_e = change_e / clen
        if change_e > 0 or prob_accept(prop_change_e, temp) > random.uniform(0, 1):
#             print('Accept')
            delta = delta_n
            eval_now = eval_new
            temp = temp * temp_ratio
        else:
            delta_rand_n = random_step(n)
            eval_rand_new = evaluate(delta_rand_n, mcmc_steps)
            k += 1
            pbar.update(1)
            change_rand_e = eval_rand_new - eval_now
            prop_random_change_e = change_rand_e / clen
            if change_e > 0 or prob_accept(prop_random_change_e, temp) > random.uniform(0, 1):
#                 print('Accept')
                delta = delta_rand_n
                eval_now = eval_rand_new
                temp = temp * temp_ratio
            else:
                temp = temp * heating_ratio
    pbar.close()
    return samp

def random_swap_step(delta, n):
    delta_ba = get_bitarray(delta)
    delta_n = delta_ba
    change_ar = n_bitarray(n)
    random.shuffle(change_ar)
    change_vals = bitarray()
    idx = 0
    for i in change_ar:
        if i:
            change_vals.append(delta_ba[idx])
        idx += 1
    random.shuffle(change_vals)
    in_idx = 0
    idx = 0
    for i in change_ar:
        if i:
            delta_n[in_idx] = change_vals[idx]
            idx += 1
        in_idx += 1
    return sq_array(delta_n)

step_swap_size = 4

# After testing for various hyperparameters, this is what we found to be the best experimentally: 
temp_initial_random = 1
temp_ratio_random = 0.8

# Simulated Annealing algorithm with random neighbour step
def simulated_anneal_random(k_max, n):
    """
    Runs a simulated annealing step
    as described in the paper/documentation
    """
    delta = random_step(n)
    samp = {}
    temp = temp_initial_random
    eval_now = evaluate(delta, mcmc_steps)
    ra_thresh = random_accept_thresh
    k = 0
    pbar = tqdm_notebook(total=k_max, desc = 'Simulated Anneal', leave = False, disable = disable_tqdm)
    while k < k_max:
        samp[eval_now] = to_N(delta)
        delta_n = random_swap_step(delta, step_swap_size)
        eval_new = evaluate(delta_n, mcmc_steps)
        k += 1
        pbar.update(1)
        change_e = eval_new - eval_now
        # we can also give the change as a proportion, which makes hyperparameter optimization
        # better for generalization (grid size does not affect what hyperparams do): 
        prop_change_e = change_e / clen
        if change_e > 0 or prob_accept(prop_change_e, temp) > random.uniform(0, 1):
#             Testing: 
#             print(eval_new)
#             print('Accept')
            delta = delta_n
            eval_now = eval_new
            temp = temp * temp_ratio_random
    pbar.close()
    return samp

`mult_simulated_anneal` runs simulated anneal multiple times. 

In [ ]:
single_run = True

def mult_simulated_anneal(times, k_max, n):
    """
    Runs simulated annealing multiple times
    """
    maxs = []
    for i in tnrange(times, desc = 'Multiple S.A.', leave = single_run, disable = disable_tqdm):
        run = simulated_anneal(k_max, n)
        maxs.append((max(run), run[max(run)]))
    return maxs

def mult_simulated_anneal_random(times, k_max, n):
    """
    Runs random simulated annealing multiple times
    """
    maxs = []
    for i in tnrange(times, desc = 'Multiple S.A. Random', leave = single_run, disable = disable_tqdm):
        run = simulated_anneal_random(k_max, n)
        maxs.append((max(run), run[max(run)]))
    return maxs

In [ ]:
samp = mult_simulated_anneal_random(samp_size, 200, 10)
for run in samp: 
    print(run)

## Testing with known cases
The following tests the various algorithms described with known cases of the grid, specifically, this only works for the $5\times 5$ case of the grid that we have known data for. It requires importing a large datafile, which is the generated datafile using `grid_search` and `analysis`. We replace the $\mathsf{Eval}$ function with a deterministic lookup (from the dataframe collected using `grid_search`), and demonstrate how well the algorithms work. 

In [ ]:
df = pickle.load(open("df.p", "rb"))

In [ ]:
def evaluate(delta, times):
    """
    Replacement for the evaulative function, which
    relies on known exhaustive searches. 
    """
    output = df.loc[to_N(delta)]['Avg']
#     print(output)
    return output

In [ ]:
samp_size = 20

### Minority Populations (Allows us to use greedy)

The following runs a multiple random sample and outputs the average maximum outcome of the optimization. 

In [ ]:
samp = mult_random_sample(samp_size, 300, 10)
sum = 0
for run in samp: 
    sum += run[0]
#     print(run)
runavg = sum / len(samp)
print()
print(runavg)

The following runs a multiple RRILS and outputs the average maximum outcome of the optimization. 

In [ ]:
samp = mult_shotgun_greedy(samp_size, 300, 10, mcmc_steps)
sum = 0
for run in samp: 
    sum += run[0]
#     print(run)
runavg = sum / len(samp)
print()
print(runavg)

The following runs a multiple simulated anneal and outputs the average maximum outcome of the optimization. 

In [ ]:
single_run = True
temp_initial = 10
temp_rate = 0.7
threshold = 0.6
samp = mult_simulated_anneal(samp_size, 300, 10)
sum = 0
for run in samp: 
    sum += run[0]
#     print(run)
runavg = sum / len(samp)
print()
print(runavg)

The following runs a multiple non-greedy simulated anneal and outputs the average maximum outcome of the optimization. 

In [ ]:
single_run = True
temp_initial = 10
temp_rate = 0.7
threshold = 0.6
samp = mult_simulated_anneal_random(samp_size, 300, 10)
sum = 0
for run in samp: 
    sum += run[0]
#     print(run)
runavg = sum / len(samp)
print()
print(runavg)

### Majority Populations (Random and modified simulated annealing only)

In [ ]:
samp = mult_random_sample(samp_size, 400, 16)
sum = 0
for run in samp: 
    sum += run[0]
#     print(run)
runavg = sum / len(samp)
print()
print(runavg)

In [ ]:
single_run = True
temp_initial = 10
temp_rate = 0.7
threshold = 0.6
samp = mult_simulated_anneal_random(samp_size, 400, 16)
sum = 0
for run in samp: 
    sum += run[0]
#     print(run)
runavg = sum / len(samp)
print()
print(runavg)

### Evaluation: Minimum time to reach certain percentile

In [ ]:
samp_size = 500
mcmc_steps = 400

The following functions finds the minimum value of $k_\mathrm{max}$ that gives a result that is as good as `bound`.

In [ ]:
def mintime_mult_random_sample(n, bound):
    k_max_required = 1
    while True: 
        samp = mult_random_sample(samp_size, k_max_required, n)
        sum = 0
        for run in samp: 
            sum += run[0]
        runavg = sum / len(samp)
        print(runavg)
        if runavg > bound: 
            return k_max_required
        else: 
            k_max_required += 10


mintime_mult_random_sample(10, 2.223665)

In [ ]:
def mintime_mult_greedy_sample(n, bound):
    k_max_required = 1
    while True: 
        samp = mult_shotgun_greedy(samp_size, k_max_required, n, mcmc_steps)
        sum = 0
        for run in samp: 
            sum += run[0]
        runavg = sum / len(samp)
        print(runavg)
        if runavg > bound: 
            return k_max_required
        else: 
            k_max_required += 50


mintime_mult_greedy_sample(10, 2.280829)

In [ ]:
def mintime_mult_sa_sample(n, bound):
    k_max_required = 1
    while True: 
        samp = mult_simulated_anneal(samp_size, k_max_required, n)
        sum = 0
        for run in samp: 
            sum += run[0]
        runavg = sum / len(samp)
        print(runavg)
        if runavg > bound: 
            return k_max_required
        else: 
            k_max_required += 50


mintime_mult_sa_sample(10, 2.280829)

In [ ]:
def mintime_mult_sarandom_sample(n, bound):
    k_max_required = 1
    while True: 
        samp = mult_simulated_anneal_random(samp_size, k_max_required, n)
        sum = 0
        for run in samp: 
            sum += run[0]
        runavg = sum / len(samp)
        print(runavg)
        if runavg > bound: 
            return k_max_required
        else: 
            k_max_required += 50


mintime_mult_sarandom_sample(10, 2.280829)

### Hyperparameter Test: 

Just come code to experiment with changing the hyperparameters (initial temperature, cooling schedule, etc) of the simulated annealing. 

In [ ]:
list = []
single_run = False

for t_i in tqdm_notebook([1, 5, 10], desc='Initial Temp'):
    for t_rate in tqdm_notebook(np.linspace(0.60, 0.99, 40), desc='Cooling Schedule', leave = False):
        for greedy_threshold in tqdm_notebook([0.6], desc='Greedy Threshold', leave = False):
            temp_initial = t_i
            temp_rate = t_rate
            threshold = greedy_threshold
            samp = mult_simulated_anneal_random(samp_size, 500, 7)
            sum = 0
            for run in samp: 
                sum += run[0]
            runavg = sum / len(samp)
            list.append([t_i, t_rate, greedy_threshold, runavg])
            print(runavg)

hp_df = pd.DataFrame(list, columns = ['t_initial', 't_rate', 'greedy_threshold', 'result'])

In [ ]:
df.loc[df['NumH'] == 8].nlargest(35, 'Avg')

In [ ]:
hp_df.nlargest(20, 'result')

In [ ]:
df.loc[df['NumH'] == 7].nlargest(35, 'Avg')

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")

sns.distplot(rand_df.loc[rand_df['k_max'] == 900]['runmax']);

In [ ]:
np.linspace(1, 100, 100).append(np.linspace(102, 150, 24))

In [ ]:
k_max_range = []
total_k_max = 0
for i in range(100): 
    k_max_range.append(i)
    total_k_max += i

for i in range(25): 
    k_max_range.append(100 + i*2)
    total_k_max += 100 + i*2
    
for i in range(25): 
    k_max_range.append(150 + i*4)
    total_k_max += 150 + i*4

for i in range(25):
    k_max_range.append(250 + i*8)
    total_k_max += 250 + i*8

for i in range(25):
    k_max_range.append(450 + i*16)
    total_k_max += 450 + i*16
    
print(k_max_range)
print(total_k_max)